In [16]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv

--2025-10-13 20:48:18--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: 'course_lead_scoring.csv.1'

     0K .......... .......... .......... .......... .......... 63% 28.1M 0s
    50K .......... .......... ........                        100% 21.9M=0.003s

2025-10-13 20:48:19 (25.4 MB/s) - 'course_lead_scoring.csv.1' saved [80876/80876]



In [17]:
import pandas as pd
import numpy as np

In [18]:
df = pd.read_csv('course_lead_scoring.csv')


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lead_source               1334 non-null   object 
 1   industry                  1328 non-null   object 
 2   number_of_courses_viewed  1462 non-null   int64  
 3   annual_income             1281 non-null   float64
 4   employment_status         1362 non-null   object 
 5   location                  1399 non-null   object 
 6   interaction_count         1462 non-null   int64  
 7   lead_score                1462 non-null   float64
 8   converted                 1462 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 102.9+ KB


In [20]:
categorical_columns = ["lead_source","industry","employment_status","location"]
numerical_columns = ["number_of_courses_viewed","annual_income","interaction_count","lead_score","converted"]

In [ ]:
for i in categorical_columns:
    print(df[i].isnull().sum())
df[categorical_columns] = df[categorical_columns].fillna("NA")


128
134
100
63


In [23]:
for i in categorical_columns:
    print(df[i].isnull().sum())

0
0
0
0


In [22]:
for i in numerical_columns:
    print(df[i].isnull().sum())
df[numerical_columns] = df[numerical_columns].fillna(0.0)

0
181
0
0
0


In [24]:
for i in numerical_columns:
    print(df[i].isnull().sum())

0
0
0
0
0


In [25]:
df['industry'].mode()

0    retail
Name: industry, dtype: object

In [26]:
matrix = df[numerical_columns].corr()
matrix

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879,0.435914
annual_income,0.009770,1.000000,0.027036,0.015610,0.053131
interaction_count,-0.023565,0.027036,1.000000,0.009888,0.374573
lead_score,-0.004879,0.015610,0.009888,1.000000,0.193673
converted,0.435914,0.053131,0.374573,0.193673,1.000000


In [27]:
from sklearn.model_selection import train_test_split

In [ ]:
target = 'converted'  
y = df[target]                   
X = df.drop(columns=[target])    


In [29]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, random_state=42
)

In [30]:
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)


In [31]:
len(X_train), len(X_val), len(X_test)


(877, 292, 293)

In [32]:
from sklearn.metrics import mutual_info_score


In [33]:
mi_scores = {}

for i in categorical_columns:
    score = mutual_info_score(X_train[i], y_train)
    mi_scores[i] = round(score, 2)


In [34]:
mi_scores = dict(sorted(mi_scores.items(), key=lambda x: x[1], reverse=True))

print(mi_scores)

{'lead_source': np.float64(0.03), 'industry': np.float64(0.02), 'employment_status': np.float64(0.02), 'location': np.float64(0.0)}


In [ ]:
train = X_train.copy()
train[target] = y_train

val = X_val.copy()
val[target] = y_val

test = X_test.copy()
test[target] = y_test

y_train = train[target]
X_train = train.drop(columns=[target])

y_val = val[target]
X_val = val.drop(columns=[target])

y_test = test[target]
X_test = test.drop(columns=[target])


In [38]:
X_train_encoded = pd.get_dummies(X_train, drop_first=True)
X_val_encoded = pd.get_dummies(X_val, drop_first=True)
X_val_encoded = X_val_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)


In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [42]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train_encoded, y_train)

y_pred = model.predict(X_val_encoded)
acc_rounded = round(accuracy_score(y_val, y_pred), 2)
print(acc_rounded)

0.74


In [43]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train_encoded, y_train)


y_pred = model.predict(X_val_encoded)
base_acc = accuracy_score(y_val, y_pred)   
print("Base accuracy:", base_acc)

Base accuracy: 0.7431506849315068


In [44]:
feature_drop_results = {}

for col in X_train_encoded.columns:
    # Drop feature col
    X_train_reduced = X_train_encoded.drop(columns=[col])
    X_val_reduced = X_val_encoded.drop(columns=[col])
    
    # Train model without this feature
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train_reduced, y_train)
    
    # Evaluate accuracy
    y_pred = model.predict(X_val_reduced)
    acc = accuracy_score(y_val, y_pred)
    
    # Accuracy drop = base - new
    diff = base_acc - acc
    feature_drop_results[col] = diff


In [45]:
sorted_results = dict(sorted(feature_drop_results.items(), key=lambda x: x[1], reverse=True))
sorted_results


{'interaction_count': 0.06849315068493145,
 'number_of_courses_viewed': 0.06506849315068486,
 'lead_score': 0.0,
 'lead_source_events': 0.0,
 'lead_source_organic_search': 0.0,
 'lead_source_referral': 0.0,
 'lead_source_social_media': 0.0,
 'industry_education': 0.0,
 'industry_finance': 0.0,
 'industry_healthcare': 0.0,
 'industry_manufacturing': 0.0,
 'industry_other': 0.0,
 'industry_retail': 0.0,
 'industry_technology': 0.0,
 'employment_status_employed': 0.0,
 'employment_status_student': 0.0,
 'location_africa': 0.0,
 'location_asia': 0.0,
 'location_australia': 0.0,
 'location_europe': 0.0,
 'location_middle_east': 0.0,
 'location_north_america': 0.0,
 'location_south_america': 0.0,
 'lead_source_paid_ads': -0.003424657534246589,
 'employment_status_self_employed': -0.003424657534246589,
 'employment_status_unemployed': -0.003424657534246589,
 'annual_income': -0.1095890410958904}

In [46]:
C_values = [0.01, 0.1, 1, 10, 100]
results = []

for C in C_values:
    model = LogisticRegression(
        solver='liblinear',
        C=C,
        max_iter=1000,
        random_state=42
    )
    model.fit(X_train_encoded, y_train)
    y_pred = model.predict(X_val_encoded)
    acc = accuracy_score(y_val, y_pred)
    results.append((C, round(acc, 3)))

results


[(0.01, 0.74), (0.1, 0.743), (1, 0.743), (10, 0.743), (100, 0.743)]